In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import csv
import time
from tqdm import tqdm

## Funzione per trasformare i dati in CSV
Lo script prende in input un file .txt con il parametro 'rb' quindi in lettura in formato bytecode che vuol dire che dovrà essere decodificato.
Per prima cosa si scorre fino alla riga che inizia con 'NR' perché i dati iniziano da li, quando la trovo imposto flag_data a 1 che serve a dirmi che da li in poi dovrò salvare le righe.
Cerco la prima riga che inizia con '0' e la salto settando firsr_row a 1.
Quando sia data_flag che first_row sono a 1 allora inizio a salvare le righe.

Per salvare le righe:
* prima le decodifico con .decode()
* poi le divido con .split() che se usato senza parametri splitta la stringa in base agli spazi
* unisco i valori delle ultime due colonne perché fanno entrambi riferimento alla data
* elimino l'ultima riga perché l'ho unita alla penultima
* faccio un append della riga


In [38]:
filename= 'DiagnosisTable4'
file1 = open(filename + ".txt", "rb")
file_csv = open(filename + '.csv', 'w+', newline ='')

count = 0
flag_data = 0
first_row = True

with file_csv:
    for line in file1:
        
        if b'\xb0C' in line:     # se trovo il carattere °C lo ignoro
            pass
        else:
            line = line.decode() # altrimenti decodifico la riga
            count = count+1      # aggiorno il contatore delle righe
            if flag_data == 1 and first_row == False:
                row = line.split()
                if not line.startswith('[END]') and len(row) == 25:
                    #print(line.split())
                    row[23] = str(row[23]) + ' ' + str(row[24])
                    del row[-1]
                    write = csv.writer(file_csv)
                    write.writerow(row)

            if line.startswith('0') and flag_data == 1:
                first_row = False

            if line.startswith('NR'):
                flag_data = 1
                #print(line)
                table_names = line.split()
                table_names.remove('LastLine')
                #print(table_names)
                write = csv.writer(file_csv)
                write.writerow(table_names)
        
print(count)

14614


## Funzione per caricare il CSV in un DataFrame

In [40]:
def load_data(file):
    column_subset = ['NR', 't_Mach', 't_HydrTa', 't_TTlubr', 't_TT_ab1', 
                     't_TT_ab2', 't_HdStUp', 't_HdStLw', 't_Sfrnt1', 
                     't_Sfrnt2', 't_Srear', 't_Smotor', 'DataTime']

    df = pd.read_csv (file,
                      usecols=column_subset,
                      index_col=False)


    df['DataTime'] = pd.to_datetime(df['DataTime'], format='%H:%M:%S %d.%m.%Y')
    
    df.set_index('DataTime', drop = True, inplace=True)
    
    #df.drop('NR', axis=1, inplace=True)
    
    return df

In [42]:
data1 = load_data('DiagnosisTable.csv')
data2 = load_data('DiagnosisTable2.csv')
data3 = load_data('DiagnosisTable3.csv')
data4 = load_data('DiagnosisTable4.csv')
#print(data1.info())
#print(data2.info())

data_all = pd.concat([data1, data2, data3, data4])
data_all.sort_index(inplace = True)
print(data_all.info())

#data1.sort_index(inplace = True)
#data1.loc['2019-06-15 19:27:01']
#data1.sort_values(by='DateTime1',ascending=True)

#print(pd.date_range(start = '2019-06-15 19:27:01', end = '2019-06-26 08:21:01' ).difference(data1.index))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 57559 entries, 2019-06-15 19:27:01 to 2020-03-30 12:33:01
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NR        57559 non-null  int64  
 1   t_Mach    57559 non-null  float64
 2   t_HydrTa  57559 non-null  float64
 3   t_TTlubr  57559 non-null  float64
 4   t_TT_ab1  57559 non-null  float64
 5   t_TT_ab2  57559 non-null  float64
 6   t_HdStUp  57559 non-null  float64
 7   t_HdStLw  57559 non-null  float64
 8   t_Sfrnt1  57559 non-null  float64
 9   t_Sfrnt2  57559 non-null  float64
 10  t_Srear   57559 non-null  float64
 11  t_Smotor  57559 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 5.7 MB
None


In [ ]:
df.plot(figsize = (20,15), grid = True)


In [ ]:
df = pd.read_csv ('parsed_data.csv', index_col=False)

for index, row in df.iterrows():
    if row['tCo_S2_Z'] != '+0':
        print(str(index) + ' ' + str(row['NR']) + ' ' + str(row['tCo_S2_Z']))
        df.drop(index, inplace=True)

fig, axs = plt.subplots(18,figsize=(15,50))
fig.suptitle('Vertically stacked subplots')
axs[0].plot(df['t_Mach'])
axs[0].set_title('t_Mach')

axs[1].plot(df['t_HydrTa'])
axs[1].set_title('t_Mach')

axs[2].plot(df[['t_TTlubr','t_TT_ab1','t_TT_ab2']])
axs[2].set_title('t_TTlubr, t_TT_ab1, t_TT_ab2')

axs[3].plot(df['t_HdStUp'])
axs[3].set_title('t_HdStUp')

axs[4].plot(df['t_HdStLw'])
axs[4].set_title('t_HdStLw')

axs[5].plot(df[['t_Sfrnt1', 't_Sfrnt2']])
axs[5].set_title('t_Sfrnt1, t_Sfrnt2')

axs[6].plot(df['t_Srear'])
axs[6].set_title('t_Srear')

axs[7].plot(df['t_Smotor'])
axs[7].set_title('t_Smotor')

axs[8].plot(df['S_Speed'])
axs[8].set_title('S_Speed')

axs[9].plot(df['S_Load'])
axs[9].set_title('S_Load')

axs[10].plot(df['tCo_H_X1'])
axs[10].set_title('tCo_H_X1')

axs[11].plot(df['tCo_H_Y1'])
axs[11].set_title('tCo_H_Y1')

axs[12].plot(df['tCo_H_Z1'])
axs[12].set_title('tCo_H_Z1')

axs[13].plot(df['tCo_H_X2'])
axs[13].set_title('tCo_H_X2')

axs[14].plot(df['tCo_H_Y2'])
axs[14].set_title('tCo_H_Y2')

axs[15].plot(df['tCo_H_Z2'])
axs[15].set_title('tCo_H_Z2')

axs[16].plot(df['tCo_H_Y3'])
axs[16].set_title('tCo_H_Y3')

axs[17].plot(df['tCo_S2_Z'])
axs[17].set_title('tCo_S2_Z')
